In [1]:
# from randtest import randtest
import numpy as np
import pandas
import pickle
from tqdm import tqdm
import random
import tensorflow.keras.backend as K

2023-11-28 23:59:04.426674: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-28 23:59:05.096391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 23:59:08.568704: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
print(cores)

80


In [3]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [4]:
def forecast_loss(y_true, y_pred):
    V=134
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

In [5]:
def score(y_preds, y_true):
    y_pred = y_preds.reshape(-1, 134)
    mmse = forecast_loss(y_true, y_pred)
    s = 0
    for i in mmse:
        s += i
    return s/len(mmse)

In [6]:
def ra_test(a_0, b_0, y_true, r=1000):
    """randomization test"""
    if len(a_0) != len(b_0):
        print('Input A and B are not eaqul-sized!')
        return
    delta_s_rs = []
    c = 0
    n = len(a_0)
    
    
    delta_s_0 = score(a_0, y_true) - score(b_0, y_true)
    delta_s_0 = delta_s_0.numpy()
    
    for _ in tqdm(range(r)):
        a_r = []
        b_r = []
        # shuffle
        for i in range(n):
            # swap
            rn = random.randint(0, 1)
            # swap
            if rn == 0:
                a_r.append(b_0[i])
                b_r.append(a_0[i])
            # do not swap
            else:
                a_r.append(a_0[i])
                b_r.append(b_0[i])

        a_r = np.array(a_r)
        b_r = np.array(b_r)
        delta_s_r = score(a_r, y_true) - score(b_r, y_true)
        delta_s_r = delta_s_r.numpy()
        # print(delta_s_r)
        # print(abs(delta_s_r))
        delta_s_rs.append(delta_s_r)
        if abs(delta_s_r) >= abs(delta_s_0):
            c += 1
        
    print(c)
    p = c/r
    print(delta_s_rs)
    return p

In [7]:
baseline_path = 'randomization_test/data/baseline.pkl'
baseline_data, var_to_ind = pickle.load(open(baseline_path, 'rb'))

forecasting_preds = []

# y_true for score
y_true = []
for y in baseline_data['forecasting_test_op']:
    y_true.append(y)
y_true = np.array(y_true)

for pred in baseline_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)
x = forecasting_preds.flatten()
del baseline_data, forecasting_preds

In [8]:
m_path = 'randomization_test/data/glove_m.pkl.pkl'
m_data, var_to_ind_m = pickle.load(open(m_path, 'rb'))
m_data

forecasting_preds = []

for pred in m_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)

y = forecasting_preds.flatten()
del m_data, forecasting_preds

In [9]:
p = ra_test(x, y, y_true = y_true, r=100)
p

2023-11-28 23:59:30.174417: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30917 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b2:00.0, compute capability: 7.0
  1%|          | 1/100 [01:22<2:15:22, 82.05s/it]

-0.010868302879808667
0.010868302879808667


  2%|▏         | 2/100 [02:44<2:14:08, 82.13s/it]

-0.014276093767723985
0.014276093767723985


  3%|▎         | 3/100 [04:05<2:12:28, 81.94s/it]

...

 99%|█████████▉| 99/100 [2:11:15<01:19, 79.08s/it]

-0.0063091858371677745
0.0063091858371677745


100%|██████████| 100/100 [2:12:34<00:00, 79.55s/it]

0.02137411230880293
0.02137411230880293
0
[-0.010868302879808667, -0.014276093767723985, 0.002904371636674341, 0.0039002501568088377, 0.014321955000912823, -0.003982132436641805, 0.008651019426435091, 0.008313311422840641, 0.00023151455420045153, -0.013564863795493487, -0.004186387077155906, -0.0203712199827768, 0.0017247100023185524, -0.006138037423351506, -0.004754504954473937, -0.005582509096220711, -0.0026962081307093655, 0.021562287463368968, -0.013479426767911917, -0.004776460518751691, 0.004617284401548183, -0.0032861592681623364, -0.01063540750997749, 0.0043844981798839555, 0.004397893063928571, -0.0007513011510438261, 0.0026024012324930013, -0.0017574663161692428, 0.006981515423752249, 0.005712857299987384, 0.006904475558986434, 0.008666175441096158, 0.00025087593986139467, 0.0011148377381369201, -0.011484224241931074, 0.001730556749672374, 0.02113300234954263, -0.006010602685282862, 0.001399314150259734, 0.0148524793481144, 0.014972840085042627, 0.0009068324502683467, 0.00691

0.0